<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Generate_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import shutil
import zipfile
import os

# Montar Google Drive
drive.mount('/content/drive')



# Definir las rutas de los archivos y directorios
source_file = '/content/drive/MyDrive/TFM/Polipos Segmentados.zip'
destination_dir = '/content/Dataset/Polipos'

# Crear el directorio si no existe
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copiar el archivo
shutil.copy(source_file, destination_dir)
print(f'Archivo copiado a: {os.path.join(destination_dir, os.path.basename(source_file))}')

# Descomprimir el archivo ZIP
zip_path = os.path.join(destination_dir, os.path.basename(source_file))
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_dir)
    print(f'Archivo descomprimido en: {destination_dir}')


Mounted at /content/drive
Archivo copiado a: /content/Dataset/Polipos/Polipos Segmentados.zip
Archivo descomprimido en: /content/Dataset/Polipos


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Definir las rutas de los archivos y directorios
source_dir = '/content/Dataset/Polipos/segmentación polipos 265'
destination_base_dir = '/content/Dataset/Polipos/TrainValTest'

# Crear directorios de destino si no existen
train_dir = os.path.join(destination_base_dir, 'train')
val_dir = os.path.join(destination_base_dir, 'val')
test_dir = os.path.join(destination_base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Listar todas las imágenes en el directorio fuente, excluyendo las que terminan en (1).jpg
all_images = [f for f in os.listdir(source_dir) if f.endswith('.jpg') and not f.endswith('(1).jpg')]

# Dividir las imágenes en train (60%), val (20%), y test (20%)
train_images, test_images = train_test_split(all_images, test_size=0.4, random_state=42)
val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)

# Función para copiar imágenes a los directorios correspondientes
def copy_images(image_list, destination_dir):
    for image in image_list:
        source_file = os.path.join(source_dir, image)
        destination_file = os.path.join(destination_dir, image)
        shutil.copyfile(source_file, destination_file)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

print("División de imágenes completada.")


División de imágenes completada.


In [5]:
import cv2
import numpy as np
import json

def find_contours(image_path):
    # Cargar la imagen
    image = cv2.imread(image_path)
    # Convertir a espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Definir el rango de color amarillo en HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    # Crear una máscara para detectar el color amarillo
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    # Encontrar contornos
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def create_annotation_from_contours(contours, image_id, filename):
    annotations = {}
    for i, contour in enumerate(contours):
        segmentation = contour.flatten().tolist()
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        annotation = {
            'shape_attributes': {
                'name': 'polygon',
                'all_points_x': [int(point[0]) for point in contour],
                'all_points_y': [int(point[1]) for point in contour]
            },
            'region_attributes': {}
        }
        annotations[str(i)] = annotation
    return {
        filename + str(image_id): {
            'fileref': '',
            'size': os.path.getsize(image_path),
            'filename': filename,
            'base64_img_data': '',
            'file_attributes': {},
            'regions': annotations
        }
    }

def process_images(image_list, source_dir, output_json_path):
    all_annotations = {}
    for image_id, image_name in enumerate(image_list):
        image_path = os.path.join(source_dir, image_name)
        contours = find_contours(image_path)
        annotations = create_annotation_from_contours(contours, image_id, image_name)
        all_annotations.update(annotations)
    with open(output_json_path, 'w') as f:
        json.dump(all_annotations, f)

# Procesar imágenes de train, val y test
process_images(train_images, train_dir, os.path.join(destination_base_dir, 'train_annotations.json'))
process_images(val_images, val_dir, os.path.join(destination_base_dir, 'val_annotations.json'))
process_images(test_images, test_dir, os.path.join(destination_base_dir, 'test_annotations.json'))

print("Generación de archivos JSON completada.")


TypeError: only length-1 arrays can be converted to Python scalars

In [10]:
import cv2
import numpy as np

def convert_yellow_to_black(image_path, output_path):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color amarillo en HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Crear una máscara para detectar el color amarillo
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la máscara
    result = np.ones_like(mask) * 255

    # Los píxeles amarillos en la máscara se convierten en negros (0) en la imagen resultante
    result[mask == 255] = 0

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0013.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba.jpg'

# Convertir los píxeles amarillos a negros y guardar la imagen
convert_yellow_to_black(image_path, output_path)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba.jpg


In [5]:
import cv2
import numpy as np

def convert_red_vertices_to_black(image_path, output_path):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color rojo en HSV
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])


    # Crear una máscara para detectar el color rojo
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mEntendido, puedo hacer eso. Aquí tienes cómo puedes hacerlo:

python

import cv2
import numpy as np

def detect_and_draw_red_vertices(image_path, output_path, distance_threshold=10):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color amarillo en HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Crear una máscara para detectar el color amarillo
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Obtener las posiciones de los píxeles amarillos
    yellow_pixels = np.argwhere(mask_yellow == 255)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask_red = cv2.inRange(hsv, lower_red, upper_red)

    # Obtener las posiciones de los píxeles rojos
    red_pixels = np.argwhere(mask_red == 255)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    # Iterar sobre los píxeles rojos y pintar en
print(f"Imagen guardada en: {output_path}")


error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'bitwise_or'
> Overload resolution failed:
>  - bitwise_or() missing required argument 'src2' (pos 2)
>  - bitwise_or() missing required argument 'src2' (pos 2)


In [8]:
import cv2
import numpy as np

def convert_custom_red_to_black(image_path, output_path):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el nuevo rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la máscara
    result = np.ones_like(mask) * 255

    # Los píxeles rojos en la máscara se convierten en negros (0) en la imagen resultante
    result[mask == 255] = 0

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0027.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba27.jpg'

# Convertir los píxeles rojos a negros y guardar la imagen
convert_custom_red_to_black(image_path, output_path)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba27.jpg


In [9]:
import cv2
import numpy as np

def detect_and_draw_red_vertices(image_path, output_path):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Encontrar los contornos de los puntos rojos
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    # Dibujar los contornos de los puntos rojos en negro (0) en la imagen resultante
    cv2.drawContours(result, contours, -1, (0, 0, 0), thickness=cv2.FILLED)

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0027.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba_V2_27.jpg'

# Detectar y dibujar los puntos rojos como vértices en negro y guardar la imagen
detect_and_draw_red_vertices(image_path, output_path)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba_V2_27.jpg


In [13]:
import cv2
import numpy as np

def detect_and_draw_red_vertices(image_path, output_path):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Aplicar una operación de dilatación para resaltar las regiones de píxeles rojos
    kernel = np.ones((5,5), np.uint8)
    dilated_mask = cv2.dilate(mask, kernel, iterations=1)

    # Aplicar una operación de erosión para reducir el tamaño de las regiones resaltadas
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)

    # Encontrar los contornos de los puntos rojos
    contours, _ = cv2.findContours(eroded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    # Dibujar los contornos de los puntos rojos en negro (0) en la imagen resultante
    cv2.drawContours(result, contours, -1, (0, 0, 0), thickness=cv2.FILLED)

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0027.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba_V3_27.jpg'

# Detectar y dibujar los puntos rojos como vértices en negro y guardar la imagen
detect_and_draw_red_vertices(image_path, output_path)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba_V3_27.jpg


In [11]:
import cv2
import numpy as np

def detect_and_draw_circular_vertices(image_path, output_path, min_radius=5, max_radius=20):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Aplicar una operación de dilatación para resaltar las regiones de píxeles rojos
    kernel = np.ones((5,5), np.uint8)
    dilated_mask = cv2.dilate(mask, kernel, iterations=1)

    # Aplicar una operación de erosión para reducir el tamaño de las regiones resaltadas
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)

    # Detectar círculos en la máscara erosionada usando la transformada de Hough para círculos
    circles = cv2.HoughCircles(eroded_mask, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=30, minRadius=min_radius, maxRadius=max_radius)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        # Iterar sobre todos los círculos detectados
        for (x, y, r) in circles:
            # Verificar si el círculo es casi redondo (radio dentro del rango específico)
            if min_radius <= r <= max_radius:
                # Dibujar el círculo en negro (0) en la imagen resultante
                cv2.circle(result, (x, y), r, (0, 0, 0), thickness=cv2.FILLED)

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0027.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba_V4_27.jpg'

# Detectar y dibujar círculos como vértices en negro y guardar la imagen
detect_and_draw_circular_vertices(image_path, output_path)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba_V4_27.jpg


In [12]:
import cv2
import numpy as np

def detect_and_draw_circular_vertices(image_path, output_path, min_radius=5, max_radius=20, dp=1, min_dist=20, param1=50, param2=30):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Aplicar una operación de dilatación para resaltar las regiones de píxeles rojos
    kernel = np.ones((5,5), np.uint8)
    dilated_mask = cv2.dilate(mask, kernel, iterations=1)

    # Aplicar una operación de erosión para reducir el tamaño de las regiones resaltadas
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)

    # Detectar círculos en la máscara erosionada usando la transformada de Hough para círculos
    circles = cv2.HoughCircles(eroded_mask, cv2.HOUGH_GRADIENT, dp=dp, minDist=min_dist, param1=param1, param2=param2, minRadius=min_radius, maxRadius=max_radius)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        # Iterar sobre todos los círculos detectados
        for (x, y, r) in circles:
            # Verificar si el círculo es casi redondo (radio dentro del rango específico)
            if min_radius <= r <= max_radius:
                # Dibujar el círculo en negro (0) en la imagen resultante
                cv2.circle(result, (x, y), r, (0, 0, 0), thickness=cv2.FILLED)

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0027.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba_V5_27.jpg'


# Detectar y dibujar círculos como vértices en negro y guardar la imagen
detect_and_draw_circular_vertices(image_path, output_path)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba_V5_27.jpg


In [14]:
import cv2
import numpy as np

def detect_and_draw_red_vertices(image_path, output_path):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask_red = cv2.inRange(hsv, lower_red, upper_red)

    # Aplicar una operación de dilatación para resaltar las regiones de píxeles rojos
    kernel = np.ones((5,5), np.uint8)
    dilated_mask_red = cv2.dilate(mask_red, kernel, iterations=1)

    # Aplicar una operación de erosión para reducir el tamaño de las regiones resaltadas
    eroded_mask_red = cv2.erode(dilated_mask_red, kernel, iterations=1)

    # Encontrar los contornos de los puntos rojos
    _, contours_red, _ = cv2.findContours(eroded_mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una máscara para detectar el color amarillo
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Detectar los bordes de color amarillo
    edges_yellow = cv2.Canny(mask_yellow, 50, 150, apertureSize=3)

    # Aplicar una operación de dilatación para resaltar los bordes
    dilated_edges_yellow = cv2.dilate(edges_yellow, kernel, iterations=1)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    # Dibujar los contornos de los puntos rojos en la imagen resultante
    cv2.drawContours(result, contours_red, -1, (0, 0, 0), thickness=cv2.FILLED)

    # Aplicar la máscara de bordes amarillos al área alrededor de los contornos rojos
    result[dilated_edges_yellow == 255] = 255

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0027.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba_Aristas_27.jpg'

# Detectar y dibujar los puntos rojos como vértices en negro y guardar la imagen
detect_and_draw_red_vertices(image_path, output_path)

print(f"Imagen guardada en: {output_path}")


ValueError: not enough values to unpack (expected 3, got 2)

In [15]:
import cv2
import numpy as np

def detect_and_draw_red_vertices(image_path, output_path, distance_threshold=10):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color amarillo en HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Crear una máscara para detectar el color amarillo
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Obtener las posiciones de los píxeles amarillos
    yellow_pixels = np.argwhere(mask_yellow == 255)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask_red = cv2.inRange(hsv, lower_red, upper_red)

    # Obtener las posiciones de los píxeles rojos
    red_pixels = np.argwhere(mask_red == 255)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    # Iterar sobre los píxeles rojos y pintar en negro si están cerca de los píxeles amarillos
    for red_pixel in red_pixels:
        for yellow_pixel in yellow_pixels:
            distance = np.linalg.norm(red_pixel - yellow_pixel)
            if distance <= distance_threshold:
                result[red_pixel[0], red_pixel[1]] = [0, 0, 0]
                break  # Salir del bucle interno si el píxel rojo está cerca de algún píxel amarillo

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0027.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba_Aristas_V2_27.jpg'

# Distancia máxima para considerar que un píxel rojo está cerca de un píxel amarillo
distance_threshold = 10

# Detectar y dibujar los puntos rojos como vértices en negro y guardar la imagen
detect_and_draw_red_vertices(image_path, output_path, distance_threshold)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba_Aristas_V2_27.jpg
